In [3]:
# Importing the necessary modules. 
import os
import csv
import requests
import time
from bs4 import BeautifulSoup
import pandas as pd

# BSN calendar pull 

1. Define a list of team numbers and names:

https://bsnpr.com/juegos/juego?cid=29382&r=BS         2022 10 35

In [8]:
# Define a list of team numbers and names

def boxscore_scrape(year, month, day):
    try:
        date_str = f"{year}{str(month).rjust(2, '0')}{str(day).rjust(2, '0')}"
        #f"{year}{month}{str(day).rjust(2, '0')}"
        url = 'https://bsnpr.com/juegos/juego?r=BS' + str(date_str)
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')

        tables = soup.findAll("table", class_="tabla_stats")
        
        if len(tables) < 2:
            print(f"No boxscore found for date {date_str}")
        else:
            t_boxscore = tables[1]
            t_hometeam = tables[2]
            t_awayteam = tables[3]
              
            folder_path = f"/Users/jeanzayas/Desktop/Divergence/Portfolio/Sports Analysis/Basketball/BSN/dataset/{year}"
            if not os.path.exists(folder_path):
                os.makedirs(folder_path)

            # extract the team names and scores from the table_boxscore
            team_names = []
            team_scores = []
            for row in t_boxscore.findAll("tr")[1:]:
                cols = row.findAll("td")
                team_names.append(cols[0].text.strip())
                team_scores.append(cols[-1].text.strip())

            # Extract the table data
            table_data = []
            for row in t_boxscore.findAll("tr")[1:]:
                cols = row.findAll("td")[1:-1]
                row_data = [col.text.strip() for col in cols]
                table_data.append(row_data)

            df_table = pd.DataFrame(table_data[:4], columns=['1Q', '2Q', '3Q', '4Q'])
            ## Check if the boxscore has at least 4 quarters
            if len(table_data) > 4:
                # Create a DataFrame from the table data with 4 columns for the first 4 quarters

                # If there are more than 4 quarters, add an 'OT' column for each additional quarter
                for i in range(5, len(table_data)):
                    column_name = f"OT{i-3}"  # Name the column 'OT5' for the 5th quarter, 'OT6' for the 6th, and so on
                    df_table[column_name] = table_data[i]

            # Add the boxscore to the list of errors if it doesn't have at least 4 quarters
            else:
                # Create a DataFrame for the team names and scores 
                df_team = pd.DataFrame({'Team': team_names, 'Total': team_scores})

                # Concatenate the two DataFrames
                df = pd.concat([df_team, df_table], axis=1)
                df.drop(2, inplace=True)
                # Extract the team names and scores from the DataFrame
                home_team = df.iloc[0]['Team']
                home_score = df.iloc[0]['Total']
                away_team = df.iloc[1]['Team']
                away_score = df.iloc[1]['Total']

                # Print the final score in the desired format
                print(f"{home_team} {home_score} v {away_team} {away_score}")

                #print(df)

            # Extract the table data fro the home team
            table_data_home = []
            for row in t_hometeam.findAll("tr")[1:]:
                cols = row.findAll("td")
                row_data = [col.text.strip() for col in cols]
                table_data_home.append(row_data)

            # Create a DataFrame for the home team stats
            df_home = pd.DataFrame(table_data_home, columns=['', 'Name', 'Min', 'M', 'A', 'FG%', 'M', 'A', '3P%', 'M', 'A', 'FT%', 'O', 'D', 'T', 'AS', 'PF', 'TO', 'ST', 'BS', 'PTS'])
            
            # Extract the table data fro the away team
            table_data_away = []
            for row in t_awayteam.findAll("tr")[1:]:
                cols = row.findAll("td")
                row_data = [col.text.strip() for col in cols]
                table_data_away.append(row_data)

            # Create a DataFrame for the away team stats
            df_away = pd.DataFrame(table_data_away, columns=['', 'Name', 'Min', 'M', 'A', 'FG%', 'M', 'A', '3P%', 'M', 'A', 'FT%', 'O', 'D', 'T', 'AS', 'PF', 'TO', 'ST', 'BS', 'PTS'])

            # Remove the first empty column
            df_home.drop(df_home.columns[0], axis=1, inplace=True)
            df_away.drop(df_away.columns[0], axis=1, inplace=True)

            # Combine the team stats into one DataFrame
            df_team_stats = pd.concat([df_home, df_away])

            # Create a DataFrame for the team names and scores 
            #df_team = pd.DataFrame({'Team': team_names, 'Total': team_scores})

            # Concatenate the three DataFrames
            #df = pd.concat([df_team, df_table, df_team_stats], axis=1)
            #df.drop(2, inplace=True)
            
          

            df.to_csv(f"/Users/jeanzayas/Desktop/Divergence/Portfolio/Sports Analysis/Basketball/BSN/dataset/{year}/{date_str}_boxscore.csv", index=False)

            df_home.to_csv(f"/Users/jeanzayas/Desktop/Divergence/Portfolio/Sports Analysis/Basketball/BSN/dataset/{year}/{date_str}_home.csv", index=False)
            df_away.to_csv(f"/Users/jeanzayas/Desktop/Divergence/Portfolio/Sports Analysis/Basketball/BSN/dataset/{year}/{date_str}_away.csv", index=False)

           
    except Exception as e:
        print(f"Error: {e}")


In [15]:
for year in range(2023, 2024):
    for month in range(10, 13):    
        for day in range(1, 100):
            try:
                boxscore_scrape(year,month,day)
                print(f"Successfully scraped boxscore for {year}-{month}-{day}")
                
            except Exception as e:
                print(f"Error: {e}, -{year}-{month}-{day}")
          

No boxscore found for date 20071001
Successfully scraped boxscore for 2007-10-1
No boxscore found for date 20071002
Successfully scraped boxscore for 2007-10-2
No boxscore found for date 20071003
Successfully scraped boxscore for 2007-10-3
No boxscore found for date 20071004
Successfully scraped boxscore for 2007-10-4
No boxscore found for date 20071005
Successfully scraped boxscore for 2007-10-5
No boxscore found for date 20071006
Successfully scraped boxscore for 2007-10-6
No boxscore found for date 20071007
Successfully scraped boxscore for 2007-10-7
No boxscore found for date 20071008
Successfully scraped boxscore for 2007-10-8
No boxscore found for date 20071009
Successfully scraped boxscore for 2007-10-9
No boxscore found for date 20071010
Successfully scraped boxscore for 2007-10-10
No boxscore found for date 20071011
Successfully scraped boxscore for 2007-10-11
No boxscore found for date 20071012
Successfully scraped boxscore for 2007-10-12
No boxscore found for date 20071013
S

KeyboardInterrupt: 

In [5]:

url = 'https://bsnpr.com/juegos/juego?r=BS20201067'
response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

tables = soup.findAll("table", class_="tabla_stats")
#extract the date, time, and game id form the first row 
date_time, game_id = [cell.text.strip() for cell in tables.findAll("th", width="200")]
date, time = date_time.split()

#create a DataFrame with the extracted data
df = pd.DataFrame({'Date': [date], 'Time': [time], 'Game ID': [game_id]})

AttributeError: ResultSet object has no attribute 'findAll'. You're probably treating a list of elements like a single element. Did you call find_all() when you meant to call find()?

In [46]:
print(tables)

[<table class="table tabla_stats" width="100%">
<tr>
<td width="25%"><img src="/images/bsn.gif" width="80"/></td>
<td align="center" width="50%"><b>BALONCESTO SUPERIOR NACIONAL<br/>
              TEMPORADA 2020</b></td>
<td align="right" width="25%"><img src="/images/aombprlogo.gif" width="80"/></td>
</tr>
</table>, <table class="table tabla_stats" width="100%">
<tr>
<th width="200">11/11/2020 3:00:00 PM</th>
<th><div align="center">1st</div></th>
<th><div align="center">2nd</div></th>
<th><div align="center">3rd</div></th>
<th><div align="center">4th</div></th>
<th><div align="center">TOTAL</div></th>
</tr>
<tr class="border_abajo_1">
<td class="score_border_abajo_r">
                    Aguada
                    </td>
<td class="score_border_abajo"><div align="center">0</div></td>
<td class="score_border_abajo"><div align="center">0</div></td>
<td class="score_border_abajo"><div align="center">0</div></td>
<td class="score_border_abajo"><div align="center">0</div></td>
<td class="sc